## 安裝套件

In [1]:
!pip install finlab > log.txt

## 營收動能

In [4]:
from finlab import data
from finlab.backtest import sim
import pandas as pd

rev = data.get("monthly_revenue:當月營收")
rev_rf = data.get("monthly_revenue:去年同月增減(%)")
vol = data.get("price:成交股數")/1000

rev_recent_3 = rev.rolling(3).sum()
vol_avg = vol.average(10)
cond1 = rev_recent_3==rev_recent_3.rolling(24, min_periods=12).max()

# 成交量條件，排除流動性過小
cond2 = vol_avg > 300

##過濾全額交割股

買進訊號隔日若為全額交割股，則不交易。
全額交割股清單每日0815更新。

In [3]:
# 過濾全額交割股
full_cash_delivery_stock_filter = data.get("etl:full_cash_delivery_stock_filter")

cond_all = cond1 & cond2 & full_cash_delivery_stock_filter 
result = rev_rf*cond_all
position = result[result>0].is_largest(10).reindex(rev.index_str_to_date().index, method="ffill")
report = sim(position=position,stop_loss=0.3,position_limit=0.1, upload=False)
report.display()

1603    0.1
2607    0.1
2734    0.1
4148    0.1
4911    0.1
4931    0.1
6541    0.1
6733    0.1
6768    0.1
8069    0.1
Name: 2022-09-12 00:00:00, dtype: float64

Timestamp('2022-09-12 00:00:00')